In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


/home/syedusmani/Documents/Projects/RAG_Assistant/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
vector_store = Chroma(embedding_function=embeddings, persist_directory="../chroma_db")

query_engine = vector_store.as_retriever(search_kwargs={"k": 4})
print("Retriever is ready. Connected to ChromaDB.")

Retriever is ready. Connected to ChromaDB.


In [8]:
test_docs = query_engine.invoke("What is gradient descent?")
print(f"Retrieved {len(test_docs)} documents.")
for doc in test_docs:
    print(f"From: {doc.metadata.get('source', 'N/A')}")
    print(doc.page_content[:200])  # First 200 chars
    print("---")

Retrieved 4 documents.
From: data/DEEP_LEARNING/Theory_08-2.pdf
# Gradient Flow  
![](data/DEEP_LEARNING/Theory_08-2.pdf-0010-01.png)  
_w w_ ∇ _w k_ +1 = _k_ − _η w_ ℒ( _k_ )  
• Gradient Descent (GD)  
_w w_ · _k_ +1 − _k_ • _w_ := _[dw]_ = −∇ _w_[G][radient Flo
---
From: data/DEEP_LEARNING/Theory_08-2.pdf
# Gradient Flow  
![](data/DEEP_LEARNING/Theory_08-2.pdf-0010-01.png)  
_w w_ ∇ _w k_ +1 = _k_ − _η w_ ℒ( _k_ )  
• Gradient Descent (GD)  
_w w_ · _k_ +1 − _k_ • _w_ := _[dw]_ = −∇ _w_[G][radient Flo
---
From: data/COMPUTER_VISION/lec14_optimizationcnn.pdf
# Gradient descent  
![](data/COMPUTER_VISION/lec14_optimizationcnn.pdf-0020-01.png)  
![](data/COMPUTER_VISION/lec14_optimizationcnn.pdf-0020-02.png)  
We want to minimize this cost function.  
CSE 1
---
From: data/COMPUTER_VISION/lec14_optimizationcnn.pdf
# Gradient descent  
![](data/COMPUTER_VISION/lec14_optimizationcnn.pdf-0020-01.png)  
![](data/COMPUTER_VISION/lec14_optimizationcnn.pdf-0020-02.png)  
We want to minimize t